In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
from fbprophet import Prophet

In [3]:
raw_train_data = pd.read_csv('drive/My Drive/digikala/1/train.csv')

In [4]:
raw_test_data = pd.read_csv('drive/My Drive/digikala/1/test.csv')
raw_test_data['sales'] = None

In [5]:
print(raw_train_data.shape)
print(raw_train_data.columns.values)
raw_train_data.tail()

(900000, 5)
['id' 'date' 'seller' 'item' 'sales']


,id,date,seller,item,sales
899995,899995,2020-04-03,6,50,11
899996,899996,2020-04-03,7,50,6
899997,899997,2020-04-03,8,50,25
899998,899998,2020-04-03,9,50,17
899999,899999,2020-04-03,10,50,16


In [6]:
print(raw_test_data.shape)
print(raw_test_data.columns.values)
raw_test_data.tail()

(13000, 5)
['id' 'date' 'seller' 'item' 'sales']


,id,date,seller,item,sales
12995,12995,2020-04-29,6,50,None
12996,12996,2020-04-29,7,50,None
12997,12997,2020-04-29,8,50,None
12998,12998,2020-04-29,9,50,None
12999,12999,2020-04-29,10,50,None


In [7]:
raw_train_data.isnull().sum()

id        0
date      0
seller    0
item      0
sales     0
dtype: int64

In [8]:
raw_test_data.isnull().sum()

id            0
date          0
seller        0
item          0
sales     13000
dtype: int64

In [9]:
raw_train_data.nunique()

id        900000
date        1800
seller        10
item          50
sales        213
dtype: int64

In [10]:
raw_test_data.nunique()

id        13000
date         26
seller       10
item         50
sales         0
dtype: int64

In [11]:
all_items = raw_train_data['item'].unique()
all_sellers = raw_train_data['seller'].unique()

In [12]:
all_item_sellers = [(item, seller) for item in all_items for seller in all_sellers]
print(len(all_item_sellers))

500


In [13]:
start = 0
while start < 500:
  temp_item_sellers = all_item_sellers[start: start + 100]
  train_data = {(item, seller): raw_train_data.loc[(raw_train_data['item']==item) & (raw_train_data['seller']==seller)] 
        for (item, seller) in temp_item_sellers}
  for (item, seller) in temp_item_sellers:
    train_data[(item, seller)] = train_data[(item, seller)].drop(['id', 'seller', 'item'], axis=1)
    train_data[(item, seller)].rename(columns = {'date':'ds', 'sales': 'y'}, inplace = True)
    train_data[(item, seller)]['ds'] = pd.to_datetime(train_data[(item, seller)]['ds'])
  models = {(item, seller): None for (item, seller) in temp_item_sellers}
  for (item, seller) in temp_item_sellers:
    model = Prophet()
    model.fit(train_data[(item, seller)])
    models[(item, seller)] = model
    print(int(start/100 + 1), item, seller)
  test_data = {(item, seller): raw_test_data.loc[(raw_test_data['item']==item) & (raw_test_data['seller']==seller)] 
        for (item, seller) in temp_item_sellers}
  for (item, seller) in temp_item_sellers:
    test_data[(item, seller)] = test_data[(item, seller)].drop(['id', 'seller', 'item', 'sales'], axis=1)
    test_data[(item, seller)].rename(columns = {'date':'ds'}, inplace = True)
    test_data[(item, seller)]['ds'] = pd.to_datetime(test_data[(item, seller)]['ds'])
  for (item, seller) in temp_item_sellers:
    test_data[(item, seller)] = models[(item, seller)].predict(test_data[(item, seller)])[['ds', 'yhat']]
    print(int(start/100 + 1), item, seller)
  for (item, seller) in temp_item_sellers:
    for date in test_data[(item, seller)]['ds']:
      index = raw_test_data.loc[(raw_test_data['item']==item) & (raw_test_data['seller']==seller) & (pd.to_datetime(raw_test_data['date'])==date)].index.values[0]
      raw_test_data.iloc[index, 4] = test_data[(item, seller)].loc[test_data[(item, seller)]['ds']==date]['yhat'].values[0]
  del models
  del train_data
  del test_data
  start += 100

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


1 1 1


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


1 1 2


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

5 50 7


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


5 50 8


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


5 50 9
5 50 10
5 41 1
5 41 2
5 41 3
5 41 4
5 41 5
5 41 6
5 41 7
5 41 8
5 41 9
5 41 10
5 42 1
5 42 2
5 42 3
5 42 4
5 42 5
5 42 6
5 42 7
5 42 8
5 42 9
5 42 10
5 43 1
5 43 2
5 43 3
5 43 4
5 43 5
5 43 6
5 43 7
5 43 8
5 43 9
5 43 10
5 44 1
5 44 2
5 44 3
5 44 4
5 44 5
5 44 6
5 44 7
5 44 8
5 44 9
5 44 10
5 45 1
5 45 2
5 45 3
5 45 4
5 45 5
5 45 6
5 45 7
5 45 8
5 45 9
5 45 10
5 46 1
5 46 2
5 46 3
5 46 4
5 46 5
5 46 6
5 46 7
5 46 8
5 46 9
5 46 10
5 47 1
5 47 2
5 47 3
5 47 4
5 47 5
5 47 6
5 47 7
5 47 8
5 47 9
5 47 10
5 48 1
5 48 2
5 48 3
5 48 4
5 48 5
5 48 6
5 48 7
5 48 8
5 48 9
5 48 10
5 49 1
5 49 2
5 49 3
5 49 4
5 49 5
5 49 6
5 49 7
5 49 8
5 49 9
5 49 10
5 50 1
5 50 2
5 50 3
5 50 4
5 50 5
5 50 6
5 50 7
5 50 8
5 50 9
5 50 10


In [14]:
# temp_item_sellers = all_item_sellers[:100]
# print(temp_item_sellers)

In [15]:
# train_data = {(item, seller): raw_train_data.loc[(raw_train_data['item']==item) & (raw_train_data['seller']==seller)] 
#         for (item, seller) in temp_item_sellers}

In [16]:
# for (item, seller) in temp_item_sellers:
#     train_data[(item, seller)] = train_data[(item, seller)].drop(['id', 'seller', 'item'], axis=1)
#     train_data[(item, seller)].rename(columns = {'date':'ds', 'sales': 'y'}, inplace = True)
#     train_data[(item, seller)]['ds'] = pd.to_datetime(train_data[(item, seller)]['ds'])

In [17]:
# models = {(item, seller): None for (item, seller) in temp_item_sellers}

In [18]:
# for (item, seller) in temp_item_sellers:
#   print((item, seller))
#   model = Prophet()
#   model.fit(train_data[(item, seller)])
#   models[(item, seller)] = model

In [19]:
# test_data = {(item, seller): raw_test_data.loc[(raw_test_data['item']==item) & (raw_test_data['seller']==seller)] 
#         for (item, seller) in temp_item_sellers}

In [20]:
# for (item, seller) in temp_item_sellers:
#     test_data[(item, seller)] = test_data[(item, seller)].drop(['id', 'seller', 'item', 'sales'], axis=1)
#     test_data[(item, seller)].rename(columns = {'date':'ds'}, inplace = True)
#     test_data[(item, seller)]['ds'] = pd.to_datetime(test_data[(item, seller)]['ds'])

In [21]:
# for (item, seller) in temp_item_sellers:
#   print((item, seller))
#   test_data[(item, seller)] = models[(item, seller)].predict(test_data[(item, seller)])[['ds', 'yhat']]

In [22]:
# for (item, seller) in temp_item_sellers:
#   for date in test_data[(item, seller)]['ds']:
#     index = raw_test_data.loc[(raw_test_data['item']==item) & (raw_test_data['seller']==seller) & (pd.to_datetime(raw_test_data['date'])==date)].index.values[0]
#     raw_test_data.iloc[index, 4] = test_data[(item, seller)].loc[test_data[(item, seller)]['ds']==date]['yhat'].values[0]

In [24]:
raw_test_data

,id,date,seller,item,sales
0,0,2020-04-04,1,1,39.0283
1,1,2020-04-04,2,1,57.9403
2,2,2020-04-04,3,1,51.1329
3,3,2020-04-04,4,1,46.0394
4,4,2020-04-04,5,1,33.0476
...,...,...,...,...,...
12995,12995,2020-04-29,6,50,18.6582
12996,12996,2020-04-29,7,50,17.1969
12997,12997,2020-04-29,8,50,29.0195
12998,12998,2020-04-29,9,50,24.2429


In [25]:
final_data = raw_test_data[['id', 'sales']]

In [26]:
final_data

,id,sales
0,0,39.0283
1,1,57.9403
2,2,51.1329
3,3,46.0394
4,4,33.0476
...,...,...
12995,12995,18.6582
12996,12996,17.1969
12997,12997,29.0195
12998,12998,24.2429


In [27]:
final_data.to_csv('prediction.csv', index=False, header=True)

In [28]:
final_data_int = final_data

In [31]:
final_data_int['sales'].map(lambda x: int(x))

0        39
1        57
2        51
3        46
4        33
         ..
12995    18
12996    17
12997    29
12998    24
12999    27
Name: sales, Length: 13000, dtype: int64

In [ ]:
final_data.to_csv('prediction.csv', index=False, header=True)